In [1]:
import CONSTANTS as c
import subprocess

In [2]:
def extract_imgs(input_video_path, output_dir, output_name, x):
    """
    use ffmpeg to extract a frame every x frames from specified video
    
    Args:
        video_path  (string): absolute path of the video
        output_dir  (string): directory where output is saved
        output_name (string): leading name of output file before frame number
        x           (int)   : extract every x'th frame
    """
    fps = 1.0 / x  #convert x to frames per second
    shell_string = 'ffmpeg -i {} -vf fps={} {}{}_%04d.png'.format(input_video_path, fps, output_dir, output_name)
    subprocess.call(shell_string, shell=True)

In [ ]:
for VID in c.VID_KEYS:
    input_video_path = c.DIR_DICT[VID][c.SRC_VID]
    output_dir = c.DIR_DICT[VID][c.IMG_DIR]
    extract_imgs(input_video_path, output_dir, VID, 10)